## Storyline: 

I) Introduction
- What is the inherent Problem with Data Soruces in Manufacturing?
- What is a Data Fabric and why does it solve that Problem?
- What is a typical Data Source in Manufacturing and how might it differ from other Data Sources?
- What is a Gateway and why am I am using Connectors (=Gateways) to access my Data?
- (0) Explain the overall Architecture

Klassische Herangehensweise mit UAExpert zeigen

II) Implementation of a Connector
- (B) Central Ontology for modelling and aligning Metadata
- Show the empty data catalog first (nothing to see there)
-> Instantiate single connector
- (C) explain the fabric mechanism

III) Decentralized Data Management
- (A) show the messaging stuff
- perform actions against a single data source based on the client
- read request and explanation
- subscribe request and explanation plus data catalog plus Kafka
- unsubscribe request and explanation plus data catalog
- read_out_data_source_and_wait plus local graph for plotting data

-> Instantiate multiple connectors (plus camera!)

IV) Knowledge Graph and Queries for centralized semantic search
- Query for the Infrastructure and show that it is visible
- Query for the full data sources and their DataPoints
- Query for active subscriptions for multiple data sources (data pipelines)
- 

V) Fully Client with topic switch
- multiple subscriptions
- coupling of multiple data sources with query action behavior (e.g. subscription)
- 

auf multi-user fähgikeit hinweisen

In [ ]:
import os
import sys
import uuid
from os import environ

import matplotlib.pyplot as plt

# Adjust depending on your folder structure — go up to project root
sys.path.append(os.path.abspath(".."))

from connector.messages.datamodel_utils import (
    SubscriptionRegisterRequest,
    SubscriptionUnregisterRequest,
)
from scripts.connector_client import ConnectorClient

from messaging.base_payload_examples import example_read_payload
from messaging.base_payload_generator import generate_read_payload

# image

<p align="center">
  <img src="../docs/img/Overview_Container_New.jpg" width="400"><br>
  <em>Figure: Overview of the container structure</em>
</p>


In [ ]:
client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])
await client.load_topic_config()
await client.return_topics()

In [ ]:
await client.switch_topic_config("assembly_station")

In [ ]:
# new data model

example_read_payload = generate_read_payload(
    nodeid_list = ["ns=2;i=59", "ns=2;i=61"]
)

print(example_read_payload.model_dump_json(indent=4))

In [ ]:
output = await client.read_data_source(base_payload=example_read_payload, timeout=0.5)

In [ ]:
subscription_id = uuid.uuid4()
print(subscription_id)
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=59"],
    device_origin="client",
    subscription_identifier=subscription_id
)

In [ ]:
output = await client.subscribe_data_source(base_payload=example_subscribe_payload)

In [ ]:
example_unsubscribe_payload = SubscriptionUnregisterRequest(
    subscription_identifier = output.root.payload.base_payload.subscription_id,
    device_origin="client"
)

In [ ]:
output = await client.unsubscribe_data_source(base_payload=example_unsubscribe_payload)

In [ ]:
await client.collect_data_from_stream(subscription_id = "92bcb013-6a16-4472-8f25-3f23186742af", duration=3)

### Real-time collection of data

In [ ]:
output = await client.collect_data_for_duration(example_subscribe_payload, duration=5)

In [ ]:
output

In [ ]:
pivot_output = output.pivot(index="timestamp", columns="nodeid", values="value")

# Plot
pivot_output.plot(figsize=(10, 6))
plt.tight_layout()
plt.show()

In [ ]:
output = await client.collect_data_for_duration_live(example_subscribe_payload, duration=5)

## SPARQL-Query

Below there is a Sparql-Query visible. We are now activly querying our data catalog for a specific data source

In [ ]:
from os import environ

from scripts.connector_client import ConnectorClient
from scripts.sparql_queries import *

SPARQL_ENDPOINT = "http://10.157.82.21:7200/repositories/new_test_repo"

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])


In [ ]:
# Nur device_identifier gegeben
q1 = build_subscription_query(device_identifier="Identifier")

# Nur service_uri gegeben
q2 = build_subscription_query(service_uri="http://example.org/service/abc")

# Nichts gegeben: allgemeine Suche
q3 = build_subscription_query()

In [ ]:
result = await client.query_graphdb(q1)
#sparql_results_to_dataframe(result)

In [ ]:
result = await client.query_graphdb(q3)

In [ ]:
device_uri = "http://stephantrattnig.org/instances#248c4256-67e0-432a-a6cf-663741bbc15b"
query = build_device_details_query(device_uri)
await client.query_graphdb([query])

In [ ]:
await client.query_graphdb(build_all_devices_query(), pretty=True)

In [ ]:
uri = "http://stephantrattnig.org/instances#248c4256-67e0-432a-a6cf-663741bbc15b"
await client.query_graphdb(build_device_details_query(uri))

In [ ]:
await client.query_graphdb(build_subscription_query(), pretty=True)

In [ ]:
from scripts.sparql_queries import *

await client.query_graphdb(build_subscription_query_new(), pretty=True)

In [ ]:
await client.query_graphdb(build_topic_query(device_identifier="CX2030"), pretty=True)
#topic_type_uri='http://stephantrattnig.org/data_fabric_ontology#requestType'

In [ ]:
await client.query_graphdb(build_all_subscriptions_query(), pretty=True)

In [ ]:
out = await client.query_graphdb(build_subscription_by_id_query("c8049abd-10b2-4397-85c5-b4a612c6f4dd"), pretty=True)

In [ ]:
out

In [ ]:
out = await client.query_graphdb(build_subscriptions_for_datapoint_query("c8049abd-10b2-4397-85c5-b4a612c6f4dd"))

In [ ]:
exemplary_query = """
PREFIX df: <http://stephantrattnig.org/data_fabric_ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?device ?deviceLabel ?connector ?connectorLabel ?topic ?topicName ?broker ?brokerAddress
WHERE {
  ?device a df:Device .
  OPTIONAL { ?device rdfs:label ?deviceLabel . }
  
  ?device df:providesService ?service .
  ?connector df:connectedTo ?service .
  OPTIONAL { ?connector rdfs:label ?connectorLabel . }
  
  ?connector df:publishesTo ?topic .
  OPTIONAL { ?topic df:topicName ?topicName . }
  
  ?broker a df:MessageBroker .
  ?broker df:hasTopic ?topic .
  OPTIONAL { ?broker df:MessageBrokerAddress ?brokerAddress . }
}
"""

exemplary_query2 = build_connector_metadata_query()

In [ ]:
await client.query_graphdb(exemplary_query2)

In [ ]:
from IPython.display import HTML, display
from pyvis.network import Network

# Create the network
net = Network(height="200px", width="100%", directed=True)
net.add_node("A", label="Device A")
net.add_node("B", label="Connector B")
net.add_edge("A", "B", label="connectedTo")

# Save the graph
net.write_html("graph.html")

# Properly display it inside Jupyter
display(HTML("graph.html"))


In [ ]:
from IPython.display import HTML, IFrame, display
from pyvis.network import Network

IFrame("graph.html", width=900, height=600)



# Helper function to shorten URIs for display
def shorten(uri):
    if "#" in uri:
        return uri.split("#")[-1]
    return uri.split("/")[-1]

# Build the PyVis network
net = Network(height="600px", width="100%", directed=True)

bindings = sparql_result["results"]["bindings"]
nodes = set()

for entry in bindings:
    s = entry["s"]["value"]
    p = entry["p"]["value"]
    o = entry["o"]["value"]

    # Add nodes (once)
    if s not in nodes:
        net.add_node(s, label=shorten(s))
        nodes.add(s)
    if o not in nodes:
        net.add_node(o, label=shorten(o))
        nodes.add(o)

    # Add edge
    net.add_edge(s, o, label=shorten(p), title=shorten(p))

# Optional: adjust layout
net.repulsion(node_distance=120, spring_length=100, spring_strength=0.01)

# Save and display
net.write_html("graph.html")
display(HTML("graph.html"))


In [ ]:
await client.query_graphdb("""
PREFIX df: <http://stephantrattnig.org/data_fabric_ontology#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT {
  ?s ?p ?o .
}
WHERE {
  ?s ?p ?o .

  {
    ?s rdf:type ?type1 .
    FILTER(?type1 IN (
      df:Device,
      df:Connector,
      df:Service,
      df:MessageBroker,
      df:Topic
    ))
  }
  UNION
  {
    ?s df:hasTopic ?any .
  }

  {
    ?o rdf:type ?type2 .
    FILTER(?type2 IN (
      df:Device,
      df:Connector,
      df:Service,
      df:MessageBroker,
      df:Topic
    ))
  }
  UNION
  {
    ?o df:hasTopic ?any2 .
  }
}

""")                           

In [ ]:
out = await client.load_topic_config()

In [ ]:
await client.switch_topic_config(module_type = None, module_id = "assembly_station")

In [ ]:
client.topic_configs

In [ ]:
client.active_topic_config